* import data

In [13]:
import pandas as pd

data = pd.read_csv('bioavailability_data_edit01.csv')
data = data.drop(columns='Unnamed: 0')

# RDKit descriptors
* add descriptor calculations

In [14]:
# we generate a list of all available metrics for view as follows
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.Descriptors import *

metric_list = Descriptors._descList

In [15]:
# create dictionary of rdkit calculations, to call the functions using their title
metric_dict = {}
for item in metric_list:
    metric_dict[item[0]] = item[1]
metric_dict.keys()

dict_keys(['MaxEStateIndex', 'MinEStateIndex', 'MaxAbsEStateIndex', 'MinAbsEStateIndex', 'qed', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons', 'MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10', 'SMR_VSA2', 'SMR_VSA3', 'SMR_VSA4', 'SMR_VSA5', 'SMR_VSA6', 'SMR_VSA7', 'SMR_VSA8', 'SMR_VSA9', 'SlogP_VSA1', 'SlogP_VSA10', 'SlogP_VS

In [16]:
# after deciding which metrics we wish to use, we'll list them here
metrics_to_add = ['MolWt','ExactMolWt','qed','MolLogP','LabuteASA','TPSA',
                  'MaxPartialCharge','MinPartialCharge','MaxAbsPartialCharge','MinAbsPartialCharge',
                  'NumHAcceptors','NumHDonors','HeavyAtomCount','NumHeteroatoms','NumRotatableBonds',
                  'NHOHCount','NOCount','FractionCSP3','RingCount','NumAliphaticRings',
                  'NumAromaticRings','NumAliphaticHeterocycles','NumAromaticHeterocycles','NumSaturatedHeterocycles',
                  'NumSaturatedRings','MolMR','BalabanJ','BertzCT','HallKierAlpha','PEOE_VSA1','PEOE_VSA2','PEOE_VSA3',
                  'PEOE_VSA4','PEOE_VSA5','PEOE_VSA6','PEOE_VSA7','PEOE_VSA8','PEOE_VSA9','PEOE_VSA10','PEOE_VSA11',
                  'PEOE_VSA12','PEOE_VSA13','PEOE_VSA14','SMR_VSA1','SMR_VSA2','SMR_VSA3','SMR_VSA4','SMR_VSA5',
                  'SMR_VSA6','SMR_VSA7','SMR_VSA8','SMR_VSA9','SMR_VSA10','SlogP_VSA1','SlogP_VSA2','SlogP_VSA3',
                  'SlogP_VSA4','SlogP_VSA5','SlogP_VSA6','SlogP_VSA7','SlogP_VSA8','SlogP_VSA9','SlogP_VSA10',
                  'SlogP_VSA11','SlogP_VSA12']

In [17]:
# now we can actually add the descriptor calculations to the data
df = data

for index,row in df.iterrows():
    drugname = df.loc[index,'Name']
    drugSMILES = df.loc[index,'smile']
    drugmol = Chem.MolFromSmiles(drugSMILES)
    
    for metric in metrics_to_add:
        function = metric_dict[metric]
        
        result = function(drugmol)
        
        df.loc[index,metric] = result

* some of the metrics we've added provide amount of the drug's surface area (SA) having a given property within a certain range
  * the full possible range of values for a property are divided into 10-14 subranges
  * the metric tells you the amount of surface area for which that property's value lies within that subrange
  * thus, if you add together all 10-14 subranges of a "VSA" metric, you get the molecule's total surface area
* I would like to test whether it would be more relevant to describe instead the percentage of SA fitting that property range
* thus, here I will create 'percent SA' alternatives for each "VSA" metric

In [18]:
# list all VSA type of metrics
VSAs = ['PEOE_VSA1','PEOE_VSA2','PEOE_VSA3','PEOE_VSA4','PEOE_VSA5','PEOE_VSA6','PEOE_VSA7',
        'PEOE_VSA8','PEOE_VSA9','PEOE_VSA10','PEOE_VSA11','PEOE_VSA12','PEOE_VSA13','PEOE_VSA14',
        'SMR_VSA1','SMR_VSA2','SMR_VSA3','SMR_VSA4','SMR_VSA5','SMR_VSA6','SMR_VSA7','SMR_VSA8',
        'SMR_VSA9','SMR_VSA10','SlogP_VSA1','SlogP_VSA2','SlogP_VSA3','SlogP_VSA4','SlogP_VSA5',
        'SlogP_VSA6','SlogP_VSA7','SlogP_VSA8','SlogP_VSA9','SlogP_VSA10','SlogP_VSA11','SlogP_VSA12']

# list just one VSA metric, which can be summed for total surface area
SMR_VSA = ['SMR_VSA1','SMR_VSA2','SMR_VSA3','SMR_VSA4','SMR_VSA5','SMR_VSA6','SMR_VSA7','SMR_VSA8','SMR_VSA9','SMR_VSA10']

for index,row in df.iterrows():
    drugname = df.loc[index,'Name']
    drugSMILES = df.loc[index,'smile']
    drugmol = Chem.MolFromSmiles(drugSMILES)
    
    # get the total SA for molecule
    totalVSA = 0
    for VSA_i in SMR_VSA: 
        totalVSA += df.loc[index,VSA_i]
    df.loc[index,'VSA_total'] = totalVSA
    
    # express all VSA metrics in Percentage of total SA
    for each_VSA in VSAs:
        # lookup absolute value of each VSA metric
        VSA_x = df.loc[index,each_VSA]
        
        # derive new name for VSA percentage column
        if each_VSA[-2].isdigit():
            num = f"{each_VSA[-2]}{each_VSA[-1]}"
        else:
            num = f"0{each_VSA[-1]}"
        titleindex = each_VSA.index('_')
        title = each_VSA[0:titleindex]
        new_column_name = 'fracVSA_'+title+num
        
        # store value of relative percent VSA for each VSA metric
        df.loc[index,new_column_name] = (VSA_x/totalVSA)

* clean up and export

In [19]:
# list of all current columns
all_columns = ['Name','smile','BA_pct','MolWt','ExactMolWt','qed','MolLogP','LabuteASA','TPSA','MaxPartialCharge','MinPartialCharge','MaxAbsPartialCharge','MinAbsPartialCharge','NumHAcceptors','NumHDonors','HeavyAtomCount','NumHeteroatoms','NumRotatableBonds','NHOHCount','NOCount','FractionCSP3','RingCount','NumAliphaticRings','NumAromaticRings','NumAliphaticHeterocycles','NumAromaticHeterocycles','NumSaturatedHeterocycles','NumSaturatedRings','MolMR','BalabanJ','BertzCT','HallKierAlpha','PEOE_VSA1','PEOE_VSA2','PEOE_VSA3','PEOE_VSA4','PEOE_VSA5','PEOE_VSA6','PEOE_VSA7','PEOE_VSA8','PEOE_VSA9','PEOE_VSA10','PEOE_VSA11','PEOE_VSA12','PEOE_VSA13','PEOE_VSA14','SMR_VSA1','SMR_VSA2','SMR_VSA3','SMR_VSA4','SMR_VSA5','SMR_VSA6','SMR_VSA7','SMR_VSA8','SMR_VSA9','SMR_VSA10','SlogP_VSA1','SlogP_VSA2','SlogP_VSA3','SlogP_VSA4','SlogP_VSA5','SlogP_VSA6','SlogP_VSA7','SlogP_VSA8','SlogP_VSA9','SlogP_VSA10','SlogP_VSA11','SlogP_VSA12','PEOE_VSA1','PEOE_VSA2','PEOE_VSA3','PEOE_VSA4','PEOE_VSA5','PEOE_VSA6','PEOE_VSA7','PEOE_VSA8','PEOE_VSA9','PEOE_VSA10','PEOE_VSA11','PEOE_VSA12','PEOE_VSA13','PEOE_VSA14','SMR_VSA1','SMR_VSA2','SMR_VSA3','SMR_VSA4','SMR_VSA5','SMR_VSA6','SMR_VSA7','SMR_VSA8','SMR_VSA9','SMR_VSA10','SlogP_VSA1','SlogP_VSA2','SlogP_VSA3','SlogP_VSA4','SlogP_VSA5','SlogP_VSA6','SlogP_VSA7','SlogP_VSA8','SlogP_VSA9','SlogP_VSA10','SlogP_VSA11','SlogP_VSA12','fracVSA_PEOE01','fracVSA_PEOE02','fracVSA_PEOE03','fracVSA_PEOE04','fracVSA_PEOE05','fracVSA_PEOE06','fracVSA_PEOE07','fracVSA_PEOE08','fracVSA_PEOE09','fracVSA_PEOE10','fracVSA_PEOE11','fracVSA_PEOE12','fracVSA_PEOE13','fracVSA_PEOE14','fracVSA_SMR01','fracVSA_SMR02','fracVSA_SMR03','fracVSA_SMR04','fracVSA_SMR05','fracVSA_SMR06','fracVSA_SMR07','fracVSA_SMR08','fracVSA_SMR09','fracVSA_SMR10','fracVSA_SlogP01','fracVSA_SlogP02','fracVSA_SlogP03','fracVSA_SlogP04','fracVSA_SlogP05','fracVSA_SlogP06','fracVSA_SlogP07','fracVSA_SlogP08','fracVSA_SlogP09','fracVSA_SlogP10','fracVSA_SlogP11','fracVSA_SlogP12','VSA_total','label1','label2','label3a','label3b']

# re-order df columns
new_col_order = ['Name','smile','BA_pct','label1','label2','label3a','label3b','MolWt','ExactMolWt','qed','MolLogP','MolMR','VSA_total','LabuteASA','TPSA','MaxPartialCharge','MinPartialCharge','MaxAbsPartialCharge','MinAbsPartialCharge','NumHAcceptors','NumHDonors','HeavyAtomCount','NumHeteroatoms','NumRotatableBonds','NHOHCount','NOCount','FractionCSP3','RingCount','NumAliphaticRings','NumAromaticRings','NumAliphaticHeterocycles','NumAromaticHeterocycles','NumSaturatedHeterocycles','NumSaturatedRings','BalabanJ','BertzCT','HallKierAlpha','PEOE_VSA1','PEOE_VSA2','PEOE_VSA3','PEOE_VSA4','PEOE_VSA5','PEOE_VSA6','PEOE_VSA7','PEOE_VSA8','PEOE_VSA9','PEOE_VSA10','PEOE_VSA11','PEOE_VSA12','PEOE_VSA13','PEOE_VSA14','SMR_VSA1','SMR_VSA2','SMR_VSA3','SMR_VSA4','SMR_VSA5','SMR_VSA6','SMR_VSA7','SMR_VSA8','SMR_VSA9','SMR_VSA10','SlogP_VSA1','SlogP_VSA2','SlogP_VSA3','SlogP_VSA4','SlogP_VSA5','SlogP_VSA6','SlogP_VSA7','SlogP_VSA8','SlogP_VSA9','SlogP_VSA10','SlogP_VSA11','SlogP_VSA12','PEOE_VSA1','PEOE_VSA2','PEOE_VSA3','PEOE_VSA4','PEOE_VSA5','PEOE_VSA6','PEOE_VSA7','PEOE_VSA8','PEOE_VSA9','PEOE_VSA10','PEOE_VSA11','PEOE_VSA12','PEOE_VSA13','PEOE_VSA14','SMR_VSA1','SMR_VSA2','SMR_VSA3','SMR_VSA4','SMR_VSA5','SMR_VSA6','SMR_VSA7','SMR_VSA8','SMR_VSA9','SMR_VSA10','SlogP_VSA1','SlogP_VSA2','SlogP_VSA3','SlogP_VSA4','SlogP_VSA5','SlogP_VSA6','SlogP_VSA7','SlogP_VSA8','SlogP_VSA9','SlogP_VSA10','SlogP_VSA11','SlogP_VSA12','fracVSA_PEOE01','fracVSA_PEOE02','fracVSA_PEOE03','fracVSA_PEOE04','fracVSA_PEOE05','fracVSA_PEOE06','fracVSA_PEOE07','fracVSA_PEOE08','fracVSA_PEOE09','fracVSA_PEOE10','fracVSA_PEOE11','fracVSA_PEOE12','fracVSA_PEOE13','fracVSA_PEOE14','fracVSA_SMR01','fracVSA_SMR02','fracVSA_SMR03','fracVSA_SMR04','fracVSA_SMR05','fracVSA_SMR06','fracVSA_SMR07','fracVSA_SMR08','fracVSA_SMR09','fracVSA_SMR10','fracVSA_SlogP01','fracVSA_SlogP02','fracVSA_SlogP03','fracVSA_SlogP04','fracVSA_SlogP05','fracVSA_SlogP06','fracVSA_SlogP07','fracVSA_SlogP08','fracVSA_SlogP09','fracVSA_SlogP10','fracVSA_SlogP11','fracVSA_SlogP12']

df_out = df[['Name','smile','BA_pct','label1','label2','label3a','label3b','MolWt','ExactMolWt','qed','MolLogP','MolMR','VSA_total','LabuteASA','TPSA','MaxPartialCharge','MinPartialCharge','MaxAbsPartialCharge','MinAbsPartialCharge','NumHAcceptors','NumHDonors','HeavyAtomCount','NumHeteroatoms','NumRotatableBonds','NHOHCount','NOCount','FractionCSP3','RingCount','NumAliphaticRings','NumAromaticRings','NumAliphaticHeterocycles','NumAromaticHeterocycles','NumSaturatedHeterocycles','NumSaturatedRings','BalabanJ','BertzCT','HallKierAlpha','PEOE_VSA1','PEOE_VSA2','PEOE_VSA3','PEOE_VSA4','PEOE_VSA5','PEOE_VSA6','PEOE_VSA7','PEOE_VSA8','PEOE_VSA9','PEOE_VSA10','PEOE_VSA11','PEOE_VSA12','PEOE_VSA13','PEOE_VSA14','SMR_VSA1','SMR_VSA2','SMR_VSA3','SMR_VSA4','SMR_VSA5','SMR_VSA6','SMR_VSA7','SMR_VSA8','SMR_VSA9','SMR_VSA10','SlogP_VSA1','SlogP_VSA2','SlogP_VSA3','SlogP_VSA4','SlogP_VSA5','SlogP_VSA6','SlogP_VSA7','SlogP_VSA8','SlogP_VSA9','SlogP_VSA10','SlogP_VSA11','SlogP_VSA12','PEOE_VSA1','PEOE_VSA2','PEOE_VSA3','PEOE_VSA4','PEOE_VSA5','PEOE_VSA6','PEOE_VSA7','PEOE_VSA8','PEOE_VSA9','PEOE_VSA10','PEOE_VSA11','PEOE_VSA12','PEOE_VSA13','PEOE_VSA14','SMR_VSA1','SMR_VSA2','SMR_VSA3','SMR_VSA4','SMR_VSA5','SMR_VSA6','SMR_VSA7','SMR_VSA8','SMR_VSA9','SMR_VSA10','SlogP_VSA1','SlogP_VSA2','SlogP_VSA3','SlogP_VSA4','SlogP_VSA5','SlogP_VSA6','SlogP_VSA7','SlogP_VSA8','SlogP_VSA9','SlogP_VSA10','SlogP_VSA11','SlogP_VSA12','fracVSA_PEOE01','fracVSA_PEOE02','fracVSA_PEOE03','fracVSA_PEOE04','fracVSA_PEOE05','fracVSA_PEOE06','fracVSA_PEOE07','fracVSA_PEOE08','fracVSA_PEOE09','fracVSA_PEOE10','fracVSA_PEOE11','fracVSA_PEOE12','fracVSA_PEOE13','fracVSA_PEOE14','fracVSA_SMR01','fracVSA_SMR02','fracVSA_SMR03','fracVSA_SMR04','fracVSA_SMR05','fracVSA_SMR06','fracVSA_SMR07','fracVSA_SMR08','fracVSA_SMR09','fracVSA_SMR10','fracVSA_SlogP01','fracVSA_SlogP02','fracVSA_SlogP03','fracVSA_SlogP04','fracVSA_SlogP05','fracVSA_SlogP06','fracVSA_SlogP07','fracVSA_SlogP08','fracVSA_SlogP09','fracVSA_SlogP10','fracVSA_SlogP11','fracVSA_SlogP12']]

# export
df_out.to_csv('bioavailability_data_edit02.csv')

* verify the saved data:

In [2]:
# reload saved data to verify
import pandas as pd
df = pd.read_csv('bioavailability_data_edit02.csv')
df = df.drop(columns='Unnamed: 0')
df.head()

,Name,smile,BA_pct,label1,label2,label3a,label3b,MolWt,ExactMolWt,qed,...,fracVSA_SlogP03,fracVSA_SlogP04,fracVSA_SlogP05,fracVSA_SlogP06,fracVSA_SlogP07,fracVSA_SlogP08,fracVSA_SlogP09,fracVSA_SlogP10,fracVSA_SlogP11,fracVSA_SlogP12
0,sulfadiazine,Nc1ccc(S(=O)(=O)Nc2ncccn2)cc1,90.0,high,very high,very high,very high,250.283,250.052447,0.787140,...,0.102152,0.000000,0.000000,0.485327,0.000000,0.000000,0.0,0.118585,0.0,0.000000
1,clofarabine,Nc1nc(Cl)nc2c1ncn2C1OC(CO)C(O)C1F,50.0,moderate,low,moderate,moderate,303.681,303.053445,0.648650,...,0.040835,0.000000,0.053688,0.054545,0.045548,0.096239,0.0,0.088002,0.0,0.100007
2,sulfamethoxazole,Cc1cc(NS(=O)(=O)c2ccc(N)cc2)no1,99.0,high,very high,very high,very high,253.283,253.052112,0.804737,...,0.102286,0.070655,0.058782,0.405645,0.000000,0.000000,0.0,0.117409,0.0,0.000000
3,tolazoline,c1ccc(CC2=NCCN2)cc1,90.0,high,very high,very high,very high,160.220,160.100048,0.691051,...,0.089738,0.000000,0.077756,0.493697,0.000000,0.000000,0.0,0.000000,0.0,0.000000
4,cotinine,CN1C(=O)CCC1c1cccnc1,97.0,high,very high,very high,very high,176.219,176.094963,0.647201,...,0.062586,0.000000,0.319123,0.320161,0.000000,0.000000,0.0,0.000000,0.0,0.000000
